In [1]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D,BatchNormalization
from tensorflow.keras.layers import Flatten, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam


In [2]:
from tensorflow.keras.models import Sequential


In [3]:
# Set constants
IMAGE_SIZE = (48, 48)
BATCH_SIZE = 512
NUM_CLASSES = 7
EPOCHS = 100

Augmentation to prevent overfitting and help with generalization

In [4]:
# Define data generators for training and validation sets
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   shear_range=0.3,
                                   zoom_range=0.3,
                                   width_shift_range=0.4,
                                   height_shift_range=0.4,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1./255)



In [5]:
train = tf.keras.utils.image_dataset_from_directory('/kaggle/input/face-expression-recognition-dataset/images/train',image_size=IMAGE_SIZE,batch_size=BATCH_SIZE,color_mode = "grayscale")
test = tf.keras.utils.image_dataset_from_directory('/kaggle/input/face-expression-recognition-dataset/images/validation',image_size=IMAGE_SIZE,batch_size=BATCH_SIZE,color_mode = "grayscale")

Found 28821 files belonging to 7 classes.
Found 7066 files belonging to 7 classes.


In [6]:
train = train.map(lambda x,y: (x/255, y))
test = test.map(lambda x,y: (x/255, y))

In [7]:
def inception_module(layer_in, f1, f2, f3):
# 1x1 conv
 conv1 = Conv2D(f1, (1,1), padding='same', activation='relu')(layer_in)
 # 3x3 conv
 conv3 = Conv2D(f2, (3,3), padding='same', activation='relu')(layer_in)
 # 5x5 conv
 conv5 = Conv2D(f3, (5,5), padding='same', activation='relu')(layer_in)
 # 3x3 max pooling
 pool = MaxPooling2D((3,3), strides=(1,1), padding='same')(layer_in)
 # concatenate filters, assumes filters/channels last
 out = Concatenate(axis=-1)([conv1, conv3, conv5, pool])
 return out

In [8]:
input_layer = Input(shape=(48,48, 1))

x = Conv2D(32,(5,5),padding='same', activation='relu')(input_layer)
x = MaxPooling2D((2,2),strides = (2,2),padding='same')(x)
x = BatchNormalization()(x)

x = Conv2D(64,(3,3),padding='same', activation='relu')(input_layer)
x = MaxPooling2D((2,2),strides = (2,2),padding='same')(x)
x = BatchNormalization()(x)

x = inception_module(x, 64, 128, 32)
x = Dropout(0.2)(x)
x = inception_module(x, 96, 192, 48)
x = Dropout(0.2)(x)
x = inception_module(x, 128, 208, 64)
x = Dropout(0.2)(x)

x = Flatten()(x)

dense = Dense(128,activation='LeakyReLU')(x)
x = Dropout(0.2)(x)
dense = Dense(128,activation='relu')(x)
out_layer = Dense(7,activation='softmax')(x)
 

model = Model(input_layer,out_layer)


In [9]:
adam =tf.keras.optimizers.Adam(
    learning_rate=0.00001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name='Adam'
    
)

In [10]:
model.compile(optimizer=adam,
               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
             metrics=['accuracy'])

print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 48, 48, 1)]  0           []                               
                                                                                                  
 conv2d_1 (Conv2D)              (None, 48, 48, 64)   640         ['input_1[0][0]']                
                                                                                                  
 max_pooling2d_1 (MaxPooling2D)  (None, 24, 24, 64)  0           ['conv2d_1[0][0]']               
                                                                                                  
 batch_normalization_1 (BatchNo  (None, 24, 24, 64)  256         ['max_pooling2d_1[0][0]']        
 rmalization)                                                                                 

In [11]:
earlystopping =tf.keras.callbacks.EarlyStopping( monitor="val_loss",
                                                patience=10, verbose=1, mode="auto", )
lr_reduce = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.6, patience=8,
                              verbose=1, mode='max', min_lr=5e-5)
checkpoint = tf.keras.callbacks.ModelCheckpoint('version1_finetune.h15', monitor= 'val_accuracy',
                             mode= 'max', save_best_only = True, verbose= 1)

In [12]:
hist = model.fit(train, epochs=EPOCHS, validation_data=test, callbacks=[earlystopping,lr_reduce,checkpoint])

Epoch 1/100


/opt/conda/lib/python3.7/site-packages/keras/backend.py:5586: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits, "Softmax", "sparse_categorical_crossentropy"
2023-02-28 12:54:33.429714: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


57/57 [==============================] - ETA: 0s - loss: 2.2036 - accuracy: 0.2338
Epoch 1: val_accuracy improved from -inf to 0.15695, saving model to version1_finetune.h15
57/57 [==============================] - 98s 1s/step - loss: 2.2036 - accuracy: 0.2338 - val_loss: 1.9295 - val_accuracy: 0.1569 - lr: 1.0000e-05
Epoch 2/100
57/57 [==============================] - ETA: 0s - loss: 1.8580 - accuracy: 0.3106
Epoch 2: val_accuracy improved from 0.15695 to 0.25318, saving model to version1_finetune.h15
57/57 [==============================] - 38s 653ms/step - loss: 1.8580 - accuracy: 0.3106 - val_loss: 1.9121 - val_accuracy: 0.2532 - lr: 1.0000e-05
Epoch 3/100
57/57 [==============================] - ETA: 0s - loss: 1.7672 - accuracy: 0.3432
Epoch 3: val_accuracy improved from 0.25318 to 0.32989, saving model to version1_finetune.h15
57/57 [==============================] - 39s 658ms/step - loss: 1.7672 - accuracy: 0.3432 - val_loss: 1.8771 - val_accuracy: 0.3299 - lr: 1.0000e-05
Epoc

In [13]:
loss, acc = model.evaluate(test, verbose=1)


14/14 [==============================] - 3s 141ms/step - loss: 1.1386 - accuracy: 0.5832


In [14]:
 model.save("Version1.h5")


In [15]:
import pandas as pd
new = pd.DataFrame.from_dict(hist.history)
new.to_csv("Version1.csv")